In [ ]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt

In [ ]:
LAMBD = 0
N_BATCH = 100
N_EPOCHS = 40

In [ ]:
def Normalization(X):
    mean_X = X.mean(axis=0)
    std_X = X.std(axis=0)
    return (X - mean_X) / std_X

In [ ]:
def LoadBatch(filename):
    with open('Dataset/'+filename, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')

        X = (dict[b"data"] / 255).T
        y = dict[b"labels"]
        Y = (np.eye(10)[y]).T

    return X, Y, y

In [ ]:
def initialization(dim):
    np.random.seed(233)
    W = np.random.normal(0,0.01,(10, dim))
    b = np.random.normal(0,0.01,(10, 1))
    return W,b

In [ ]:
def EvaluateClassifier(X,W,b):
    results = np.dot(W,X)+b
    softmax = np.exp(results) / np.sum(np.exp(results), axis=0)
    return (softmax)

In [ ]:
def ComputeCost(X, Y, W, b, lambd):
        N = X.shape[1]

        P = EvaluateClassifier(X, W, b)
        cost = 1/N * - np.sum(Y*np.log(P)) + lambd * np.sum(W**2)

        return cost

In [ ]:
def ComputeAccuracy(y, P):
    preds = np.argmax(P,axis=0).T
    diff = preds - y
    num_correct = diff[diff == 0].shape[0]
    num_tot = np.array(y).shape[0]
    acc = num_correct / float(num_tot)
    return acc

In [ ]:
def ComputeGradients(X, Y, P, W, lambd):
        N = X.shape[1]
        G = - (Y - P)
        grad_W = 1 / N * G@X.T + 2 * lambd * W
        grad_b = np.reshape(1 / N * G@np.ones(N), (Y.shape[0], 1))
        return grad_W, grad_b

In [ ]:
def ComputeGradsNum(X, Y, P, W, b, lamda, h = 1e-6):
    """ Converted from matlab code """
    no = W.shape[0]
    d = X.shape[0]

    grad_W = np.zeros(W.shape);
    grad_b = np.zeros((no, 1));

    c = ComputeCost(X, Y, W, b, lamda);

    for i in range(len(b)):
        b_try = np.array(b)
        b_try[i] += h
        c2 = ComputeCost(X, Y, W, b_try, lamda)
        grad_b[i] = (c2-c) / h

    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            W_try = np.array(W)
            W_try[i,j] += h
            c2 = ComputeCost(X, Y, W_try, b, lamda)
            grad_W[i,j] = (c2-c) / h

    return [grad_W, grad_b]

In [ ]:
def ComputeGradsNumSlow(X, Y, P, W, b, lamda, h = 1e-6):
    """ Converted from matlab code """
    no = W.shape[0]
    d = X.shape[0]

    grad_W = np.zeros(W.shape);
    grad_b = np.zeros((no, 1));

    for i in range(len(b)):
        b_try = np.array(b)
        b_try[i] -= h
        c1 = ComputeCost(X, Y, W, b_try, lamda)

        b_try = np.array(b)
        b_try[i] += h
        c2 = ComputeCost(X, Y, W, b_try, lamda)

        grad_b[i] = (c2-c1) / (2*h)

    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            W_try = np.array(W)
            W_try[i,j] -= h
            c1 = ComputeCost(X, Y, W_try, b, lamda)

            W_try = np.array(W)
            W_try[i,j] += h
            c2 = ComputeCost(X, Y, W_try, b, lamda)

            grad_W[i,j] = (c2-c1) / (2*h)

    return [grad_W, grad_b]

In [ ]:
def Separate_mini_batch(Xtrain,Ytrain):
    Xbatches = []
    Ybatches = []
    for j in range(1,int(Xtrain.shape[1]/N_BATCH)+1):
        start = (j-1)*N_BATCH
        end = j*N_BATCH
        Xbatch = Xtrain[:,start:end]
        Ybatch = Ytrain[:,start:end]
        Xbatches.append(Xbatch)
        Ybatches.append(Ybatch)
    return Xbatches,Ybatches

In [ ]:
def train_network(Eta):
    X,Y,y = LoadBatch('data_batch_1')
    X = Normalization(X)
    W,b = initialization(X.shape[0])
    Xtest, Ytest, ytest = LoadBatch('test_batch')
    Xtest = Normalization(Xtest)
    Xbatches, Ybatches = Separate_mini_batch(X,Y)
    

    costs_train = []
    costs_validation = []

    for epoch in range(0,N_EPOCHS):
        for batch in range(0,len(Xbatches)):

            P = EvaluateClassifier(Xbatches[batch],W,b)

            gradW, gradB = ComputeGradients(Xbatches[batch],Ybatches[batch],P,W,LAMBD)

            W = W - Eta*gradW
            b = b - Eta*gradB
        # Eta = Eta * 0.9
        
        P_validation = EvaluateClassifier(Xtest,W,b)
        P_train = EvaluateClassifier(X,W,b)
        
        cost_train = ComputeCost(X,Y,W,b,LAMBD)
        cost_validation = ComputeCost(Xtest,Ytest,W,b,LAMBD)
        
        acc_train = ComputeAccuracy(y,P_train)
        acc_validation = ComputeAccuracy(ytest,P_validation)
        
        print("Epoch: %d" %(epoch+1))
        print("Training Cost: %f" %cost_train)
        print("Validation Cost: %f" %cost_validation)
        print("Training Accuracy: %f" %acc_train)
        print("Validation Accuracy: %f" %acc_validation)
        print("---------------------------------")
        
        costs_train.append(cost_train)
        costs_validation.append(cost_validation)

    return costs_validation,costs_train,W,b

In [ ]:
def check_gradients(X,Y,P,W,b,lambd):
        gradW_an, gradb_an = ComputeGradients(
            X[:, :2], Y[:, :2], P, W, lambd)
        gradW_nu, gradb_nu = ComputeGradsNum(
            X[:, :2], Y[:, :2], P, W, b, lambd)
        gradW_nu_sl, gradb_nu_sl = ComputeGradsNumSlow(
            X[:, :2], Y[:, :2], P, W, b, lambd)
        np.testing.assert_almost_equal(gradW_an, gradW_nu, decimal=6)
        np.testing.assert_almost_equal(gradb_an, gradb_nu, decimal=6)
        
        np.testing.assert_almost_equal(gradW_an, gradW_nu_sl, decimal=6)
        np.testing.assert_almost_equal(gradb_an, gradb_nu_sl, decimal=6)

In [ ]:
def test_grad():
    X,Y,y = LoadBatch('data_batch_1')
    W,b = initialization(X.shape[0])
    Xbatches, Ybatches = Separate_mini_batch(X,Y)
    try:
        for batch in range(0,len(Xbatches)):
            P = EvaluateClassifier(Xbatches[batch][:, :2],W,b)
            check_gradients(Xbatches[batch], Ybatches[batch], P, W, b, LAMBD)
            
    except:
        print("Gradients can work.")

In [ ]:
def montage(W):
    """ Display the image for each label in W """
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(2,5)
    for i in range(2):
        for j in range(5):
            im  = W[i*5+j,:].reshape(32,32,3, order='F')
            sim = (im-np.min(im[:]))/(np.max(im[:])-np.min(im[:]))
            sim = sim.transpose(1,0,2)
            ax[i][j].imshow(sim, interpolation='nearest')
            ax[i][j].set_title("y="+str(5*i+j))
            ax[i][j].axis('off')
    plt.show()

In [ ]:
def compute_loss(costs_train,costs_validation):
    loss_train=[]
    loss_validation=[]
    for i in range(len(costs_train)):
        loss_train.append(sum(costs_train[0:i+1])/(i+1))
        loss_validation.append(sum(costs_validation[0:i+1])/(i+1))
    return loss_train, loss_validation

In [ ]:
def plot_cost(costs_train,costs_validation):
    epochs_arr = np.arange(0, N_EPOCHS).tolist()

    plt.plot(epochs_arr, costs_train, 'r-',label='training cost')
    plt.plot(epochs_arr, costs_validation, 'g-',label='validation cost')
    plt.legend(loc='upper center', shadow=True)
    plt.xlabel('Epoch')
    plt.ylabel('Cost')
    plt.show()

In [ ]:
def plot_loss(costs_train,costs_validation):
    epochs_arr = np.arange(0, N_EPOCHS).tolist()

    plt.plot(epochs_arr, costs_train, 'r-',label='training loss')
    plt.plot(epochs_arr, costs_validation, 'g-',label='validation loss')
    plt.legend(loc='upper center', shadow=True)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

In [ ]:
# test_grad()

In [ ]:
costs_validation,costs_train,W,b = train_network(0.001)

In [ ]:
montage(W)

In [ ]:
plot_cost(costs_train,costs_validation)

In [ ]:
loss_train, loss_validation = compute_loss(costs_train,costs_validation)

In [ ]:
plot_loss(loss_train, loss_validation)